# Google Colab Notebook for YOLOv11 Emotion Detection Model

## Step 1: Set up the environment

In [ ]:
!nvidia-smi  # Verify GPU availability

Wed Jul 16 16:59:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install ultralytics # Install required packages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import yaml

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive

Mounted at /content/drive


## Step 2: Upload and unzip dataset from device

In [ ]:
# Check if ZIP file exists and unzip
zip_path = "/content/facial-expression-in-classroom-12.zip"
if not os.path.exists(zip_path):
    raise FileNotFoundError(f"Dataset ZIP file not found at {zip_path}. Please upload it to Colab.")
!unzip -q {zip_path} -d /content

## Step 3: Verify and set dataset path

In [ ]:
# Device uploads to /content/facial-expression-in-classroom-12
dataset_path = "/content/facial-expression-in-classroom-12"
train_img_dir = os.path.join(dataset_path, "train", "images")
train_label_dir = os.path.join(dataset_path, "train", "labels")

In [ ]:
# Check if train directories exist
if not os.path.exists(train_img_dir) or not os.path.exists(train_label_dir):
    raise FileNotFoundError(f"Train directories not found at {train_img_dir} or {train_label_dir}")

## Step 4: Create validation set

In [ ]:
# Split train into train and val
val_img_dir = os.path.join(dataset_path, "valid", "images")
val_label_dir = os.path.join(dataset_path, "valid", "labels")
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

In [ ]:
# Get list of image files
img_files = [f for f in os.listdir(train_img_dir) if f.endswith(".jpg")]
if not img_files:
    raise FileNotFoundError(f"No .jpg files found in {train_img_dir}")

train_imgs, val_imgs = train_test_split(img_files, test_size=0.2, random_state=42)

In [ ]:
# Move validation images and labels
for img in val_imgs:
    src_img = os.path.join(train_img_dir, img)
    dst_img = os.path.join(val_img_dir, img)
    shutil.move(src_img, dst_img)
    label_file = img.replace(".jpg", ".txt")
    src_label = os.path.join(train_label_dir, label_file)
    dst_label = os.path.join(val_label_dir, label_file)
    if os.path.exists(src_label):
        shutil.move(src_label, dst_label)
    else:
        print(f"Warning: Label file {src_label} not found for {img}")

## Step 5: Update data.yaml with absolute paths

In [ ]:
data_yaml_path = os.path.join(dataset_path, "data.yaml")
with open(data_yaml_path, "r") as f:
    data_yaml = yaml.safe_load(f)

# Use absolute paths to avoid path resolution issues
data_yaml["train"] = os.path.abspath(os.path.join(dataset_path, "train/images"))
data_yaml["val"] = os.path.abspath(os.path.join(dataset_path, "valid/images"))
data_yaml["test"] = ""  # No test set available
data_yaml["nc"] = 7
data_yaml["names"] = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_yaml, f)

In [ ]:
# Verify paths exist
if not os.path.exists(data_yaml["train"]):
    raise FileNotFoundError(f"Train image directory not found: {data_yaml['train']}")
if not os.path.exists(data_yaml["val"]):
    raise FileNotFoundError(f"Validation image directory not found: {data_yaml['val']}")

## Step 6: Load and train YOLOv11 model

In [ ]:
model = YOLO("yolo11s.pt")  # Load pretrained YOLOv11 small model

100%|██████████| 18.4M/18.4M [00:00<00:00, 108MB/s] 


In [ ]:
# Train the model
model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    batch=32,
    device=0,  # Use T4 GPU
    patience=5,
    optimizer="AdamW",
    lr0=0.001,
    name="yolo11_emotion_detection"
)

Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/facial-expression-in-classroom-12/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11_emotion_detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=5, perspectiv

100%|██████████| 755k/755k [00:00<00:00, 29.7MB/s]

Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

 23        [16, 19, 22]  1    822117  ultralytics.nn.modules.head.Detect           [7, [128, 256, 512]]          
YOLO11s summary: 181 layers, 9,430,501 parameters, 9,430,485 gradients, 21.6 GFLOPs

Transferred 493/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 156MB/s]


...

                   all        233       2769      0.902      0.932      0.966      0.883



50 epochs completed in 0.414 hours.
Optimizer stripped from runs/detect/yolo11_emotion_detection/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/yolo11_emotion_detection/weights/best.pt, 19.2MB

Validating runs/detect/yolo11_emotion_detection/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,415,509 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:06<00:00,  1.53s/it]


                   all        233       2769      0.901      0.933      0.967      0.882
                 Angry        211        404      0.896      0.908      0.962      0.878
               Disgust        131        141      0.863      0.938      0.957      0.869
                  Fear         92        113      0.869      0.877      0.934      0.859
                 Happy        222        381      0.903      0.982       0.98      0.903
               Neutral        233       1220      0.931      0.944       0.98      0.902
                   Sad        149        248      0.947      0.944       0.98      0.882
              Surprise        205        262      0.901      0.935      0.973      0.884
Speed: 0.5ms preprocess, 3.9ms inference, 0.0ms loss, 5.5ms postprocess per image
Results saved to runs/detect/yolo11_emotion_detection


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d0ceaed4710>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

## Step 7: Save training results to Google Drive

In [ ]:
training_results_dir = "runs/detect/yolo11_emotion_detection"
drive_save_path = "/content/drive/MyDrive/[Sem. 7 & 8] Seminar & Skripsi/Program/Model/YOLOv11s_Emotion_Detection/yolo11_emotion_detection_results"

In [ ]:
# Create the save directory in Google Drive if it doesn't exist
os.makedirs(drive_save_path, exist_ok=True)

In [ ]:
# Copy the training results to Google Drive
shutil.copytree(training_results_dir, os.path.join(drive_save_path, "yolo11_emotion_detection"), dirs_exist_ok=True)
print(f"Training results saved to {drive_save_path}/yolo11_emotion_detection")

Training results saved to /content/drive/MyDrive/[Sem. 7 & 8] Seminar & Skripsi/Program/Model/YOLOv11s_Emotion_Detection/yolo11_emotion_detection_results/yolo11_emotion_detection


## Step 8: Batch evaluation and inference

In [ ]:
# Perform validation (mAP) and inference on the validation set in one go
val_results = model.val(
    data=data_yaml_path,
    batch=32,  # Process validation in batches
    conf=0.25,  # Confidence threshold for inference
    save=True,  # Save inference results
    split="val"  # Use validation set
)

# Extract and print mAP metrics
print("Validation Metrics:")
print(f"mAP@50: {val_results.box.map50:.4f}")
print(f"mAP@50:95: {val_results.box.map:.4f}")

# Inference results are saved in the 'runs/detect' directory
print(f"Batch inference results saved in 'runs/detect/yolo11_emotion_detection' directory")

Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2575.1±1365.1 MB/s, size: 228.9 KB)


val: Scanning /content/facial-expression-in-classroom-12/valid/labels.cache... 233 images, 0 backgrounds, 0 corrupt: 100%|██████████| 233/233 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:07<00:00,  1.13it/s]


                   all        233       2769      0.902      0.932      0.958      0.894
                 Angry        211        404      0.897      0.906      0.946      0.884
               Disgust        131        141      0.863      0.937       0.95      0.873
                  Fear         92        113      0.869      0.877      0.924      0.886
                 Happy        222        381      0.903      0.982      0.979      0.919
               Neutral        233       1220      0.931      0.944       0.97      0.909
                   Sad        149        248      0.949      0.944      0.975      0.893
              Surprise        205        262      0.903      0.935      0.964      0.893
Speed: 4.5ms preprocess, 5.4ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs/detect/yolo11_emotion_detection3
Validation Metrics:
mAP@50: 0.9584
mAP@50:95: 0.8941
Batch inference results saved in 'runs/detect/yolo11_emotion_detection' directory


## Step 9: Save inference results to Google Drive

In [ ]:
inference_results_dir = "runs/detect/yolo11_emotion_detection2"
drive_inference_save_path = os.path.join(drive_save_path, "inference_results")
shutil.copytree(inference_results_dir, drive_inference_save_path, dirs_exist_ok=True)
print(f"Inference results saved to {drive_inference_save_path}")

Inference results saved to /content/drive/MyDrive/[Sem. 7 & 8] Seminar & Skripsi/Program/Model/YOLOv11s_Emotion_Detection/yolo11_emotion_detection_results/inference_results


## Step 10: Export the model (optional)

In [ ]:
model.export(format="onnx")  # Export to ONNX for deployment
print("Model exported to ONNX format")

Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from 'runs/detect/yolo11_emotion_detection/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 11, 8400) (18.3 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.59', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 5.8s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.59...
ONNX: export success ✅ 8.7s, saved as 'runs/detect/yolo11_emotion_detection/weights/best.onnx' (36.2 MB)

Export complete (9.5s)
Results saved to /content/runs/detect/yolo11_emotion_detection/weights
Predict:         yolo predict task=detect model=runs/detect

In [ ]:
# Save exported model to Google Drive
exported_model_path = "runs/detect/yolo11_emotion_detection/weights/best.onnx"
drive_exported_model_path = os.path.join(drive_save_path, "best.onnx")
shutil.copy(exported_model_path, drive_exported_model_path)
print(f"Exported ONNX model saved to {drive_exported_model_path}")

Exported ONNX model saved to /content/drive/MyDrive/[Sem. 7 & 8] Seminar & Skripsi/Program/Model/YOLOv11s_Emotion_Detection/yolo11_emotion_detection_results/best.onnx
